In [18]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [19]:
import pandas as pd
import numpy as np

In [20]:
article_sentiment_path = "/content/drive/My Drive/weekly_sentiment_lag.csv"
article_sentiment = pd.read_csv(article_sentiment_path)

market_data_path = "/content/drive/My Drive/Master.csv"
market_data = pd.read_csv(market_data_path)

pd.set_option('display.max_columns',1000)
pd.set_option('display.width',10000)

print(article_sentiment)
print(market_data)

     Start Date    End Date  Sentiment Score  sentiment_weighted_lag_1  sentiment_weighted_lag_2  sentiment_weighted_lag_3
0    2016-01-04  2016-01-10         0.505057                  0.454551                  0.409096                  0.368186
1    2016-01-11  2016-01-17         0.443757                  0.454551                  0.409096                  0.368186
2    2016-01-18  2016-01-24         0.556918                  0.399381                  0.409096                  0.368186
3    2016-01-25  2016-01-31         0.425082                  0.501226                  0.359443                  0.368186
4    2016-02-01  2016-02-07         0.505660                  0.382574                  0.451103                  0.323499
..          ...         ...              ...                       ...                       ...                       ...
341  2024-10-21  2024-10-27         0.528765                  0.470155                  0.481904                  0.328837
342  2024-10-28 

In [21]:


market_data.set_index('Date', inplace=True)

for col in ['GDP USD', 'GDP CAD', 'Interest Rate USD', 'Interest Rate CAD', 'CPI USD', 'CPI CAD']:
    for lag in range(1, 4):  # Lag 1 to 3 quarters
        market_data[f'{col}_Lag_{lag}'] = market_data[col].shift(lag * 90)

market_data.reset_index(inplace=True)
market_data.bfill(inplace=True)

print(market_data)


            Date     Open     High      Low    Close   GDP USD   GDP CAD  Interest Rate USD Interest Rate CAD  CPI USD  CPI CAD  GDP USD_Lag_1  GDP USD_Lag_2  GDP USD_Lag_3  GDP CAD_Lag_1  GDP CAD_Lag_2  GDP CAD_Lag_3  Interest Rate USD_Lag_1  Interest Rate USD_Lag_2  Interest Rate USD_Lag_3 Interest Rate CAD_Lag_1 Interest Rate CAD_Lag_2 Interest Rate CAD_Lag_3  CPI USD_Lag_1  CPI USD_Lag_2  CPI USD_Lag_3  CPI CAD_Lag_1  CPI CAD_Lag_2  CPI CAD_Lag_3
0     2016-01-03  1.38737  1.39400  1.38480  1.38645  0.579519  0.591684               0.34              0.50  237.652    126.8       0.579519       0.579519       0.579519       0.591684       0.591684       0.591684                     0.34                     0.34                     0.34                    0.50                    0.50                    0.50        237.652        237.652        237.652          126.8          126.8          126.8
1     2016-01-04  1.39363  1.39817  1.38960  1.39343  0.579519  0.591684               0.3

In [22]:


sentiment_columns = ['Sentiment Score', 'sentiment_weighted_lag_1', 'sentiment_weighted_lag_2', 'sentiment_weighted_lag_3']
for col in sentiment_columns:
    market_data[col] = None

for i, row in market_data.iterrows():
    date = row['Date']
    # Find the corresponding sentiment row
    matching_row = article_sentiment[(article_sentiment['Start Date'] <= date) & (article_sentiment['End Date'] >= date)]
    if not matching_row.empty:
        for col in sentiment_columns:
            market_data.at[i, col] = matching_row.iloc[0][col]

market_data.bfill(inplace=True)

print(market_data)

save_directory = "/content/drive/My Drive/CombinedFinal.csv"
market_data.to_csv(save_directory, index=False)

            Date     Open     High      Low    Close   GDP USD   GDP CAD  Interest Rate USD Interest Rate CAD  CPI USD  CPI CAD  GDP USD_Lag_1  GDP USD_Lag_2  GDP USD_Lag_3  GDP CAD_Lag_1  GDP CAD_Lag_2  GDP CAD_Lag_3  Interest Rate USD_Lag_1  Interest Rate USD_Lag_2  Interest Rate USD_Lag_3 Interest Rate CAD_Lag_1 Interest Rate CAD_Lag_2 Interest Rate CAD_Lag_3  CPI USD_Lag_1  CPI USD_Lag_2  CPI USD_Lag_3  CPI CAD_Lag_1  CPI CAD_Lag_2  CPI CAD_Lag_3  Sentiment Score  sentiment_weighted_lag_1  sentiment_weighted_lag_2  sentiment_weighted_lag_3
0     2016-01-03  1.38737  1.39400  1.38480  1.38645  0.579519  0.591684               0.34              0.50  237.652    126.8       0.579519       0.579519       0.579519       0.591684       0.591684       0.591684                     0.34                     0.34                     0.34                    0.50                    0.50                    0.50        237.652        237.652        237.652          126.8          126.8          1

<ipython-input-22-e278b1dfa86b>:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  market_data.bfill(inplace=True)
